# COVID19 Detection

---




In [ ]:
#Import drive files
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%tensorflow_version 2.x #Tensorflow version

In [ ]:
!nvidia-smi #GPU

## Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Input, Model

from utils.metrics import *
from utils.utils import *
from utils.preprocess import *
from utils.gradcam import compute_gradcam

## Segmentation models

To do the segmentation task it is commonly used the U-Net model

In [ ]:
def Unet_3():
  inputs = Input(shape=(224, 224, 1), name='input')
  #s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)

  #Contraction path
  c1 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs)
  c1 = tf.keras.layers.Dropout(0.1)(c1)
  c1 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
  p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

  c2 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
  c2 = tf.keras.layers.Dropout(0.1)(c2)
  c2 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
  p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
  
  c3 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
  c3 = tf.keras.layers.Dropout(0.2)(c3)
  c3 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
  p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)
  
  c4 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
  c4 = tf.keras.layers.Dropout(0.2)(c4)
  c4 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
  p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)
  
  c5 = tf.keras.layers.Conv2D(1024, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
  c5 = tf.keras.layers.Dropout(0.3)(c5)
  c5 = tf.keras.layers.Conv2D(1024, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

  #Expansive path 
  u6 = tf.keras.layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c5)
  u6 = tf.keras.layers.concatenate([u6, c4])
  c6 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
  c6 = tf.keras.layers.Dropout(0.2)(c6)
  c6 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
  
  u7 = tf.keras.layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c6)
  u7 = tf.keras.layers.concatenate([u7, c3])
  c7 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
  c7 = tf.keras.layers.Dropout(0.2)(c7)
  c7 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
  
  u8 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c7)
  u8 = tf.keras.layers.concatenate([u8, c2])
  c8 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
  c8 = tf.keras.layers.Dropout(0.1)(c8)
  c8 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
  
  u9 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c8)
  u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
  c9 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
  c9 = tf.keras.layers.Dropout(0.1)(c9)
  c9 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
  
  outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)
  
  model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
  model.compile(optimizer='adam', loss=dice_coef_loss, metrics=['accuracy',dice_coef,iou,])
  model.summary()
  return model

## Classification models

In [ ]:
#VGG16 for 1-channel images with imagent weights

def get_model_VGG16_gray():
    model = tf.keras.applications.VGG16(weights='imagenet', include_top=False)

    # Block1_conv1 weights are of the format [3, 3, 3, 64] -> this is for RGB images
    # For grayscale, format should be [3, 3, 1, 64]. Weighted average of the features has to be calculated across channels.
    # RGB weights: Red 0.2989, Green 0.5870, Blue 0.1140

    # getting weights of block1 conv1.
    block1_conv1 = model.get_layer('block1_conv1').get_weights()
    weights, biases = block1_conv1

    # :weights shape = [3, 3, 3, 64] - (0, 1, 2, 3)
    # convert :weights shape to = [64, 3, 3, 3] - (3, 2, 0, 1)
    weights = np.transpose(weights, (3, 2, 0, 1))


    kernel_out_channels, kernel_in_channels, kernel_rows, kernel_columns = weights.shape

    # Dimensions : [kernel_out_channels, 1 (since grayscale), kernel_rows, kernel_columns]
    grayscale_weights = np.zeros((kernel_out_channels, 1, kernel_rows, kernel_columns))

      # iterate out_channels number of times
    for i in range(kernel_out_channels):

        # get kernel for every out_channel
        get_kernel = weights[i, :, :, :]

        temp_kernel = np.zeros((3, 3))

        # :get_kernel shape = [3, 3, 3]
        # axis, dims = (0, in_channel), (1, row), (2, col)

        # calculate weighted average across channel axis
        in_channels, in_rows, in_columns = get_kernel.shape

        for in_row in range(in_rows):
            for in_col in range(in_columns):
                feature_red = get_kernel[0, in_row, in_col]
                feature_green = get_kernel[1, in_row, in_col]
                feature_blue = get_kernel[2, in_row, in_col]

            # weighted average for RGB filter
            total = (feature_red * 0.2989) + (feature_green * 0.5870) + (feature_blue * 0.1140)

            temp_kernel[in_row, in_col] = total


        # :temp_kernel is a 3x3 matrix [rows x columns]
        # add an axis at the end to specify in_channel as 1

        # Second: Add axis at the start of :temp_kernel to make its shape: [1, 3, 3] which is [in_channel, rows, columns]
        temp_kernel = np.expand_dims(temp_kernel, axis=0)

        # Now, :temp_kernel shape is [1, 3, 3]

        # Concat :temp_kernel to :grayscale_weights along axis=0
        grayscale_weights[i, :, :, :] = temp_kernel

      # Dimension of :grayscale_weights is [64, 1, 3, 3]
      # In order to bring it to tensorflow or keras weight format, transpose :grayscale_weights

      # dimension, axis of :grayscale_weights = (out_channels: 0), (in_channels: 1), (rows: 2), (columns: 3)
      # tf format of weights = (rows: 0), (columns: 1), (in_channels: 2), (out_channels: 3)

      # Go from (0, 1, 2, 3) to (2, 3, 1, 0)
    grayscale_weights = np.transpose(grayscale_weights, (2, 3, 1, 0)) # (3, 3, 1, 64)

      # combine :grayscale_weights and :biases
    new_block1_conv1 = [grayscale_weights, biases]


      # Reconstruct the layers of VGG16 but replace block1_conv1 weights with :grayscale_weights

      # get weights of all the layers starting from 'block1_conv2'
    vgg16_weights = {}
    for layer in model.layers[2:]:
        if "conv" in layer.name:
            vgg16_weights["224_" + layer.name] = model.get_layer(layer.name).get_weights()

    del model


      # Custom build VGG16
    input = Input(shape=(224, 224, 1), name='224_input')
      # Block 1
    x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 1), data_format="channels_last", name='224_block1_conv1')(input)
    x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='224_block1_conv2')(x)
    x = tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2), name='224_block1_pool')(x)

      # Block 2
    x = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same', name='224_block2_conv1')(x)
    x = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same', name='224_block2_conv2')(x)
    x = tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2), name='224_block2_pool')(x)

      # Block 3
    x = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='224_block3_conv1')(x)
    x = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='224_block3_conv2')(x)
    x = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='224_block3_conv3')(x)
    x = tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2), name='224_block3_pool')(x)

      # Block 4
    x = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='224_block4_conv1')(x)
    x = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='224_block4_conv2')(x)
    x = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='224_block4_conv3')(x)
    x = tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2), name='224_block4_pool')(x)

      # Block 5
    x = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='224_block5_conv1')(x)
    x = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='224_block5_conv2')(x)
    x = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='224_block5_conv3')(x)
    x = tf.keras.layers.MaxPooling2D((8, 8), strides=(8, 8), name='224_block5_pool')(x)

    base_model = Model(inputs=input, outputs=x)

    base_model.get_layer('224_block1_conv1').set_weights(new_block1_conv1)
    for layer in base_model.layers[2:]:
        if 'conv' in layer.name:
            base_model.get_layer(layer.name).set_weights(vgg16_weights[layer.name])

    x = base_model.output

    for layer in base_model.layers:
        layer.trainable = True

    x = tf.keras.layers.GlobalAveragePooling2D()(x)  
    layers = tf.keras.layers.Flatten()(x)
      #layers = tf.keras.layers.Dense(128,activation="relu")(layers)
    layers = tf.keras.layers.Dropout(0.2)(layers)
    layers = tf.keras.layers.Dense(1024 ,activation="relu")(layers)
    layers = tf.keras.layers.Dropout(0.2)(layers)
    layers = tf.keras.layers.Dense( 512,activation="relu")(layers)
    layers = tf.keras.layers.Dense( 64,activation="relu")(layers)
    predictions = tf.keras.layers.Dense(2, activation="softmax", name="output_1")(layers)

      #Compilador
    model = tf.keras.Model(inputs = base_model.input, outputs=predictions)
    optimizer=tf.keras.optimizers.Adam(lr=0.0001) 
    loss='categorical_crossentropy'
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
    model.summary()
    return model
##################################################################################################################################################

#VGG19 for 1-channel images with imagent weights

def get_model_VGG19_gray():
      model = tf.keras.applications.VGG19(weights='imagenet', include_top=False)

      # Block1_conv1 weights are of the format [3, 3, 3, 64] -> this is for RGB images
      # For grayscale, format should be [3, 3, 1, 64]. Weighted average of the features has to be calculated across channels.
      # RGB weights: Red 0.2989, Green 0.5870, Blue 0.1140

      # getting weights of block1 conv1.
      block1_conv1 = model.get_layer('block1_conv1').get_weights()
      weights, biases = block1_conv1

      # :weights shape = [3, 3, 3, 64] - (0, 1, 2, 3)
      # convert :weights shape to = [64, 3, 3, 3] - (3, 2, 0, 1)
      weights = np.transpose(weights, (3, 2, 0, 1))


      kernel_out_channels, kernel_in_channels, kernel_rows, kernel_columns = weights.shape

      # Dimensions : [kernel_out_channels, 1 (since grayscale), kernel_rows, kernel_columns]
      grayscale_weights = np.zeros((kernel_out_channels, 1, kernel_rows, kernel_columns))

      # iterate out_channels number of times
      for i in range(kernel_out_channels):

        # get kernel for every out_channel
        get_kernel = weights[i, :, :, :]

        temp_kernel = np.zeros((3, 3))

        # :get_kernel shape = [3, 3, 3]
        # axis, dims = (0, in_channel), (1, row), (2, col)

        # calculate weighted average across channel axis
        in_channels, in_rows, in_columns = get_kernel.shape

        for in_row in range(in_rows):
          for in_col in range(in_columns):
            feature_red = get_kernel[0, in_row, in_col]
            feature_green = get_kernel[1, in_row, in_col]
            feature_blue = get_kernel[2, in_row, in_col]

            # weighted average for RGB filter
            total = (feature_red * 0.2989) + (feature_green * 0.5870) + (feature_blue * 0.1140)

            temp_kernel[in_row, in_col] = total


        # :temp_kernel is a 3x3 matrix [rows x columns]
        # add an axis at the end to specify in_channel as 1

        # Second: Add axis at the start of :temp_kernel to make its shape: [1, 3, 3] which is [in_channel, rows, columns]
        temp_kernel = np.expand_dims(temp_kernel, axis=0)

        # Now, :temp_kernel shape is [1, 3, 3]

        # Concat :temp_kernel to :grayscale_weights along axis=0
        grayscale_weights[i, :, :, :] = temp_kernel

      # Dimension of :grayscale_weights is [64, 1, 3, 3]
      # In order to bring it to tensorflow or keras weight format, transpose :grayscale_weights

      # dimension, axis of :grayscale_weights = (out_channels: 0), (in_channels: 1), (rows: 2), (columns: 3)
      # tf format of weights = (rows: 0), (columns: 1), (in_channels: 2), (out_channels: 3)

      # Go from (0, 1, 2, 3) to (2, 3, 1, 0)
      grayscale_weights = np.transpose(grayscale_weights, (2, 3, 1, 0)) # (3, 3, 1, 64)

      # combine :grayscale_weights and :biases
      new_block1_conv1 = [grayscale_weights, biases]


      # Reconstruct the layers of VGG16 but replace block1_conv1 weights with :grayscale_weights

      # get weights of all the layers starting from 'block1_conv2'
      vgg19_weights = {}
      for layer in model.layers[2:]:
        if "conv" in layer.name:
          vgg19_weights["224_" + layer.name] = model.get_layer(layer.name).get_weights()

      del model


      # Custom build VGG19
      input = Input(shape=(224, 224, 1), name='224_input')
      # Block 1
      x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 1), data_format="channels_last", name='224_block1_conv1')(input)
      x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='224_block1_conv2')(x)
      x = tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2), name='224_block1_pool')(x)

      # Block 2
      x = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same', name='224_block2_conv1')(x)
      x = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same', name='224_block2_conv2')(x)
      x = tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2), name='224_block2_pool')(x)

      # Block 3
      x = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='224_block3_conv1')(x)
      x = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='224_block3_conv2')(x)
      x = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='224_block3_conv3')(x)
      x = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='224_block3_conv4')(x)
      x = tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2), name='224_block3_pool')(x)

      # Block 4
      x = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='224_block4_conv1')(x)
      x = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='224_block4_conv2')(x)
      x = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='224_block4_conv3')(x)
      x = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='224_block4_conv4')(x)
      x = tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2), name='224_block4_pool')(x)

      # Block 5
      x = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='224_block5_conv1')(x)
      x = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='224_block5_conv2')(x)
      x = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='224_block5_conv3')(x)
      x = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='224_block5_conv4')(x)
      x = tf.keras.layers.MaxPooling2D((8, 8), strides=(8, 8), name='224_block5_pool')(x)

      base_model = Model(inputs=input, outputs=x)

      base_model.get_layer('224_block1_conv1').set_weights(new_block1_conv1)
      for layer in base_model.layers[2:]:
        if 'conv' in layer.name:
          base_model.get_layer(layer.name).set_weights(vgg19_weights[layer.name])

      x = base_model.output

      for layer in base_model.layers:
          layer.trainable = True

      x = tf.keras.layers.GlobalAveragePooling2D()(x)  
      layers = tf.keras.layers.Flatten()(x)
      #layers = tf.keras.layers.Dense(128,activation="relu")(layers)
      layers = tf.keras.layers.Dropout(0.2)(layers)
      layers = tf.keras.layers.Dense(1024 ,activation="relu")(layers)
      layers = tf.keras.layers.Dropout(0.2)(layers)
      layers = tf.keras.layers.Dense( 512,activation="relu")(layers)
      layers = tf.keras.layers.Dense( 64,activation="relu")(layers)
      predictions = tf.keras.layers.Dense(2, activation="softmax", name="output_1")(layers)

      #Compilador
      model = tf.keras.Model(inputs = base_model.input, outputs=predictions)
      optimizer=tf.keras.optimizers.Adam(lr=0.0001) 
      model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=['accuracy'])
      model.summary()
      return model

## Import Databases and training

The images were saved in npy format, so that the loading of the images is faster and more efficient. 

The shape is (224,224) in order to apply transfer learning

In [ ]:
PATH='' #Base PATH 

### Databases for segmentation

To train the segmentation models is necessary the image (X) and a binary mask (y)

In [ ]:
X_train = np.load(PATH+'/DATABASES/SEGMENTATION/X_M+J+N_train.npy')
X_dev = np.load(PATH+'/DATABASES/SEGMENTATION/X_M+J+N_dev.npy')
Y_train = np.load(PATH+'/DATABASES/SEGMENTATION/y_M+J+N_train.npy')
Y_dev = np.load(PATH+'/DATABASES/SEGMENTATION/y_M+J+N_dev.npy')

In [ ]:
print('Images Shape is:\nTrain:', X_train.shape,'\nDev: ',X_dev.shape)
print('\nMasks Shape:\nTrain: ',Y_train.shape,'\nDev: ',Y_dev.shape)

In [ ]:
index=2 #Change to view different images

plt.figure(figsize=(10, 10))
plt.subplot(121)
plt.title("Original")
plt.axis('off')
plt.imshow(X_train[index], cmap='gray')
plt.subplot(122)
plt.title("Mask")
plt.axis('off')
plt.imshow(Y_train[index], cmap='gray')

plt.show()

#### Preprocess

In [ ]:
#Convert to float32 to avoid error of multiplication
Y_train = Y_train.astype('float32')
Y_dev = Y_dev.astype('float32')

#Min-max Normalization
X_train,Y_train=min_max_preprocessing(X_train,Y_train)
X_dev,Y_dev=min_max_preprocessing(X_dev,Y_dev)


#Z-score Normalization (mean and standard deviation)
X_train,Y_train,mean,std=samplewise_preprocessing(X_train,Y_train)
X_dev=featurewise_preprocessing(X_dev,mean,std) # Mean and std of train data is used

#### Train segmentation 

In [ ]:
model=Unet_3()

In [ ]:
epochs=3 #Change if you want to train more

history=model.fit(X_train, Y_train, epochs=3, batch_size=32, validation_data=(X_dev, Y_dev)) 

In [ ]:
plot_graphs(history,'dice_coef')
plot_graphs(history,'iou')
plot_graphs(history,'loss')
plot_graphs(history,'accuracy')

In [ ]:
# Load weights from previous trained model (200 epochs)
model.load_weights(PATH+'/WEIGHTS/Unet_3_M+J+N_1.h5')

In [ ]:
#Predict mask
Pred_Masks = model.predict(X_dev)

#Resize
pred_img=[]
for i in Pred_Masks:
  pred_img.append(np.resize(i,(224,224)))

#Apply binary
thresh= 0.3
pred = []
for a in pred_img:
  a[a >= thresh] = 1
  a[a < thresh] = 0
  pred.append(a)


#Multiply masks by the original images
Seg_imgs=pred*X_dev

In [ ]:
#Plot results
index=60 #Change to view different images

plt.figure(figsize=(10, 10))
plt.subplot(141)
plt.title("Original")
plt.axis('off')
plt.imshow(X_dev[index], cmap='gray')
plt.subplot(142)
plt.title("Real Mask")
plt.axis('off')
plt.imshow(Y_dev[index], cmap='gray')
plt.subplot(143)
plt.title("Predicted Mask")
plt.axis('off')
plt.imshow(pred[index], cmap='gray')
plt.subplot(144)
plt.title("Segmented Image")
plt.axis('off')
plt.imshow(Seg_imgs[index], cmap='gray')

plt.show()

### Databases for classification

The data is saved as normal images and also as previously lung-segmented images

**Note:** Choose if you want train on **Normal Images** or **Segmented Images** and run that cell

#### Normal (Without Segmentation)

In [ ]:
#Positive COVID19 images
X_train_P = np.load(PATH+'/DATABASES/POSITIVE/NORMAL/X_pos_train_C.npy')[:1400] #Only a portion of the dataset for the code to run without problem in colab can run.
X_test_P = np.load(PATH+'/DATABASES/POSITIVE/NORMAL/X_pos_test_C.npy')[:300] #Only a portion of the dataset for the code to run without problem in colab can run.
X_dev_P = np.load(PATH+'/DATABASES/POSITIVE/NORMAL/X_pos_dev_C.npy')[:300] #Only a portion of the dataset for the code to run without problem in colab can run.


#Negative COVID19 images
X_train_N = np.load(PATH+'/DATABASES/NEGATIVE/NORMAL/X_train_COVID_neg2.npy')[:1400]
X_test_N = np.load(PATH+'/DATABASES/NEGATIVE/NORMAL/X_test_COVID_neg2.npy')[:300]
X_dev_N = np.load(PATH+'/DATABASES/NEGATIVE/NORMAL/X_dev_COVID_neg2.npy')[:300]

#### Segmented Images

In [ ]:
#Positive COVID19 images
X_train_P = np.load(PATH+'/DATABASES/POSITIVE/SEGMENTED/X_pos_Seg_Train.npy')[:1400] #Only a portion of the dataset for the code to run without problem in colab can run.
X_test_P = np.load(PATH+'/DATABASES/POSITIVE/SEGMENTED/X_pos_Seg_Test.npy')[:300] #Only a portion of the dataset for the code to run without problem in colab can run.
X_dev_P = np.load(PATH+'/DATABASES/POSITIVE/SEGMENTED/X_pos_Seg_Dev.npy')[:300] #Only a portion of the dataset for the code to run without problem in colab can run.


#Negative COVID19 images
X_train_N = np.load(PATH+'/DATABASES/NEGATIVE/SEGMENTED/X_preC_Seg_Train.npy')[:1400]
X_test_N = np.load(PATH+'/DATABASES/NEGATIVE/SEGMENTED/X_preC_Seg_Test.npy')[:300]
X_dev_N = np.load(PATH+'/DATABASES/NEGATIVE/SEGMENTED/X_preC_Seg_Dev.npy')[:300]

#### Information of the Database

In [ ]:
print('Positive Images Shape:\nTrain:',X_train_P.shape,'\nDev: ',X_dev_P.shape,'\nTest: ',X_test_P.shape)
print('\nNegative Images Shape:\nTrain:',X_train_N.shape,'\nDev: ',X_dev_N.shape,'\nTest: ',X_test_N.shape)

In [ ]:
"""##Preprocess""" 

#Create Labels
y_train_N=np.array([[1,0]]*len(X_train_N))
y_dev_N=np.array([[1,0]]*len(X_dev_N))
y_test_N=np.array([[1,0]]*len(X_test_N))

y_train_P=np.array([[0,1]]*len(X_train_P))
y_dev_P=np.array([[0,1]]*len(X_dev_P))
y_test_P=np.array([[0,1]]*len(X_test_P))


#Concateneta Positive and Negative
X_train=np.concatenate((X_train_N,X_train_P))
X_dev=np.concatenate((X_dev_N,X_dev_P))
X_test=np.concatenate((X_test_N,X_test_P))
y_train=np.concatenate((y_train_N,y_train_P))
y_dev=np.concatenate((y_dev_N,y_dev_P))
y_test=np.concatenate((y_test_N,y_test_P))


#X_train=np.array(X_train,dtype=np.float64)
#X_dev=np.array(X_dev,dtype=np.float64)
#X_test=np.array(X_test,dtype=np.float64)



print('Imagenes entrenamiento: ',len(X_train))
print('Imagenes validacion: ',len(X_dev))
print('Imagenes test: ',len(X_test))

#Min_max Normalization
X_train,y_train=min_max_preprocessing(X_train,y_train)
X_dev,y_dev=min_max_preprocessing(X_dev,y_dev)
X_test,y_test=min_max_preprocessing(X_test,y_test)


#Z-Score Normalization (mean and std)
X_train,y_train,mean,std=samplewise_preprocessing(X_train,y_train)
X_dev=featurewise_preprocessing(X_dev,mean,std)
X_test=featurewise_preprocessing(X_test,mean,std)


#X_train,X_dev,X_test=convert_n_CH(X_train,X_dev,X_test,mode='rgb')

print('Imagenes entrenamiento procesadas: ',len(X_train))
print('Imagenes validacion procesadas: ',len(X_dev))
print('Imagenes test procesadas: ',len(X_test))


In [ ]:
index=0 #Change to show other images

plt.axis('off')
plt.imshow(X_train[index], cmap='gray')
plt.colorbar()
plt.show()

#### Train Classification

In [ ]:
#Select model - Uncomment the model you want 
model = get_model_VGG19_gray()
#model= get_model_VGG16_gray

In [ ]:
epochs=3 #Change for the number of epochs you want

history=model.fit(X_train, y_train, epochs=epochs, batch_size=64, validation_data=(X_dev, y_dev))

In [ ]:
plot_graphs(history,'loss')
plot_graphs(history,'accuracy')

Load the correct weights, it depends of the model (VGG16 or VGG19), and the type of data you trained (Normal or Segmented)

**Note: For correct performance it's important to import the whole dataset.**

In [ ]:
#Normal
#VGG16
#model.load_weights(PATH+'/WEIGHTS/VGG16_imagenet_posypre_normal.h5')
#VGG19
#model.load_weights(PATH+'/WEIGHTS/VGG19_imagenet_posypre_normal.h5')


#Segmented
#VGG16
#model.load_weights(PATH+'/WEIGHTS/VGG16_imagenet_posypre_seg.h5')
#VGG19
model.load_weights(PATH+'/WEIGHTS/VGG19_imagenet_posypre_seg.h5')

In [ ]:
#Metrics of classification

predictions=model.predict(np.expand_dims(X_test,axis=3))
y_pred_bool = np.argmax(predictions, axis=-1)
y_hat=np.argmax(y_test,axis=-1)

metrics(y_hat,y_pred_bool)

In [ ]:
tf.compat.v1.disable_eager_execution() #To plot GradCAM (Heatmaps) is necessary to disable eager execution and import the model again

model = get_model_VGG19_gray()


#Normal
#VGG16
#model.load_weights(PATH+'/WEIGHTS/VGG16_imagenet_posypre_normal.h5')
#VGG19
#model.load_weights(PATH+'/WEIGHTS/VGG19_imagenet_posypre_normal.h5')


#Segmented
#VGG16
#model.load_weights(PATH+'/WEIGHTS/VGG16_imagenet_posypre_seg.h5')
#VGG19
model.load_weights(PATH+'/WEIGHTS/VGG19_imagenet_posypre_seg.h5')

In [ ]:
index=320
labels=['No-COVID','COVID']
compute_gradcam(model,np.expand_dims(np.expand_dims(X_test[index], axis=0),axis=3), labels,layer_name='224_block5_conv4')

In [ ]:
#ROC Curve
get_roc_curve(y_test, predictions, labels)

In [ ]:
#Precision Recall Curve
get_prc_curve(y_test, predictions, labels)